In [30]:
import pandas as pd
import numpy as np
import json

file_path = '/home/mantovani/Documents/IC/HMC_TE-s/Temp_Data/Dados_Dominios_Conservados/sequences.json'

df = {'Name':                   [],
      'Nucleotide_Sequence':    []}

with open(file_path, mode='r') as file:

    dados = json.load(file)

    for i in dados['results']:

        df['Name'].append(i['crossReferences'][0]['name'].strip()[0:11])
        df['Nucleotide_Sequence'].append(i['sequence'])
        
        l_reading_frames = len(i['openReadingFrames'])

        for j in range(l_reading_frames):

            reading_frame = i['openReadingFrames'][j]

            if f'Start_{j}' not in df:

                df[f'Start_{j}'] = []
                df[f'End_{j}'] = []
                df[f'Strand_{j}'] = []
                df[f'Protein_Sequence_{j}'] = []

            df[f'Start_{j}'].append(reading_frame['start'])
            df[f'End_{j}'].append(reading_frame['end'])
            df[f'Strand_{j}'].append(reading_frame['strand'])
            df[f'Protein_Sequence_{j}'].append(reading_frame['protein']['sequence'])

        for j in range(l_reading_frames, (len(df) - 1) // 4):  # Dividimos por 4 pois criamos 4 colunas por reading frame, subtraimos 1, pois desconsideramos o 'Name'

            df[f'Start_{j}'].append(np.nan)
            df[f'End_{j}'].append(np.nan)
            df[f'Strand_{j}'].append(np.nan)
            df[f'Protein_Sequence_{j}'].append(np.nan)

df = pd.DataFrame(df)
df.head()

,Name,Nucleotide_Sequence,Start_0,End_0,Strand_0,Protein_Sequence_0,Start_1,End_1,Strand_1,Protein_Sequence_1,...,Strand_5,Protein_Sequence_5,Start_6,End_6,Strand_6,Protein_Sequence_6,Start_7,End_7,Strand_7,Protein_Sequence_7
0,DF000000001,acagtatagcatagtggttaagagcacgggctctggagccagactg...,21,119,ANTISENSE,GTERLSNLPKVTQLVSGRAGIRTQAVWLQSPCS,22,135,SENSE,EHGLWSQTAWVRIPALPLTSCVTLGKLLNLSVPQFPHL,...,SENSE,QYSIVVKSTGSGARLPGFESRLCHLLAV,167,250,ANTISENSE,HLLSAYYVPGTVLSALHVLTHLILTTTL,181,261,SENSE,GLNELIHVKRLEQCLAHSKRSINVSYY
1,DF000000006,tctatctatataaaatgcttaggtatgtttccccaaggttcagctg...,627,788,ANTISENSE,TYRKLSSFEYERHICIRWNERYARYENSIQVHCSAHSCNTYFLCYA...,373,672,ANTISENSE,HIFPMLCSSTVDYSLLIIRIRYFYLISPRRFALFLLISNRSSLSSM...,...,SENSE,AFIYLIKGLKVLRSVKMAEPCVRTLRNRRRNANETLEERSNRLRDR...,175,336,SENSE,ELYAFLSIYLLNLFEIXLRISTTCNVKHVSPILNLIQCIKVFIRIV...,641,871,ANTISENSE,LNTGFGRYKQILSKVCPCDLLMVIEKAKRIGNCLLLNMKGIFVSDG...
2,DF000000004,cagtcatgcgccgcataacgacgtttcggtcaacgacggaccgcat...,251,331,SENSE,VCVSTLYDVRTTTKSPNDAFLRTYPRR,108,221,ANTISENSE,AIWYSLLLLGYKPVQHVTVLNTVGNCNTMVSICVSKHI,...,ANTISENSE,VMRRLTTGIRSEKCVVRRFRRCANIIECTYTNLDGIAYYTPRLYGI...,2,115,SENSE,SHAPHNDVSVNDGPHIRRWSHKIIIPYFYCTFSMFRYV,75,194,SENSE,YRIFTVPFLCLDMFRYTNTYHCVTIAYSIQYSNMLYRFVA
3,DF000000008,atggtagatttaaacccaancatatcaataattacattaaatgtaa...,543,851,ANTISENSE,YVVFSXSFSSKYFLISXVISSLTHGLFRSVLFNFQIFGDFLXIFLL...,3,311,ANTISENSE,IDPFIIMKCPSLSLVIFLALKSTLSDINIATPAFFXLVFAWYIFFH...,...,ANTISENSE,XVEHLFMCLLAIXISSFVKCLFKSFAHFXIGLFVFXLLXCRSSLYI...,2617,2898,ANTISENSE,VYDPSXINFCVWCEVGVEXHFFPYGYPIVPAPFVEKTILSPLNCXG...,1902,2351,ANTISENSE,FSVXRSCTXFVRFIPKYXMFFDAIVNGIVFXISFSNCLLLVYRNTI...
4,DF000000007,ggccgggcgcggtggctcacgcctgtaatcccagcactttgggagg...,229,312,SENSE,AVIAPLHSSLGDRARPCLKKKKKKKKKK,3,311,ANTISENSE,FFFFFFFFFLRQGLALSPRLECSGAITAHCSLDLPGSSDPPASASR...,...,ANTISENSE,FFFFFFFFFFETGSRSVAQAGVQWRDHGSLQPRPPGLKRSSRLSLP...,204,311,SENSE,AREVEAAVSRDRATALQPGRQSETLSQKKKKKKKKK,1,225,SENSE,GRARWLTPVIPALWEAEAGGSLEPRSSRPAWATWRNPVSTKNTKIS...


In [27]:
Aminoacido_Nucleotidio = {
    'F': ('UUU', 'UUC'),
    'L': ('UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'),
    'I': ('AUU', 'AUC', 'AUA'),
    'M': ('AUG'),
    'V': ('GUU', 'GUC', 'GUA', 'GUG'),
    'S': ('UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'),
    'P': ('CCU', 'CCC', 'CCA', 'CCG'),
    'T': ('ACU', 'ACC', 'ACA', 'ACG'),
    'A': ('GCU', 'GCC', 'GCA', 'GCG'),
    'Y': ('UAU', 'UAC'),
    'H': ('CAU', 'CAC'),
    'Q': ('CAA', 'CAG'),
    'N': ('AAU', 'AAC'),
    'K': ('AAA', 'AAG'),
    'D': ('GAU', 'GAC'),
    'E': ('GAA', 'GAG'),
    'C': ('UGU', 'UGC'),
    'W': ('UGG'),
    'R': ('CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'),
    'G': ('GGU', 'GGC', 'GGA', 'GGG'),
    '*': ('UAA', 'UAG', 'UGA')  # Códons de parada
}

def grafo_N(nulceotidio: str) -> list:

    nulceotidio:str = nulceotidio.upper()

    if nulceotidio == 'A':

        return [1,0,0,0,0]
    
    elif nulceotidio == 'C':

        return [0,1,0,0,0]
    
    elif nulceotidio == 'G':

        return [0,0,1,0,0]
    
    elif nulceotidio == 'U':

        return [0,0,0,1,0]
    
    else: 

        return [0,0,0,0,1]
    

def Proteina_trinca(proteina: str) -> np.ndarray:

    def Aminoacido_Grafo(aminoacido: str) -> np.ndarray:

        A = [np.array([grafo_N(j) for j in i]) for i in Aminoacido_Nucleotidio.get(aminoacido, [])]
        
        codon_array = np.logical_or.reduce(A)

        return codon_array

    proteina = proteina.upper()

    B = [Aminoacido_Grafo(i) for i in proteina.strip()]

    return np.vstack(B).astype(int)

def Sequencia_grafo(sequencia: str) -> np.ndarray:

    a = [grafo_N(i) for i in sequencia.strip()]

    return np.array(a)

In [29]:
print('proteina: \n', Proteina_trinca('GTERLSNLPKVTQLVSGRAGIRTQAVWLQSPCS'))

print('\n', Sequencia_grafo("acagtatagcatagtggttaagagcacgggctctggagccagactgcctgggttcgaatcccggctctgccacttactagctgtgtgaccttgggcaagttacttaacctctctgtgcctcagtttcctcatctgtaaaatggggataataatagtacctacctcatagggttgttgtgaggattaaatgagttaatacatgtaaagcgcttagaacagtgcctggcacatagtaagcgctcaataaatgttagctattatt"))


 [[1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 ...
 [1 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [ ]:
def agrupar_tensor(linha: pd.DataFrame)